In [ ]:
import pandas as pd
import airportsdata
airports = airportsdata.load('ICAO')
import pandas as pd
path_fold_save = "../data/final/race_bar/"
path_21 = "preprocessing/divergence/raw_data/2021/*.csv.gz"
path_22 = "preprocessing/divergence/raw_data/2022/*.csv.gz"

In [ ]:
iso_codes = [
    "GL",  # BG - Groenlandia
    "IS",  # BI - Islanda
    "XK",  # BK - Kosovo
    None,  # E  - Generico Europa Nord
    "BE",  # EB - Belgio
    "DE",  # ED - Germania
    "EE",  # EE - Estonia
    "FI",  # EF - Finlandia
    "GB",  # EG - Regno Unito
    "NL",  # EH - Paesi Bassi
    "IE",  # EI - Irlanda
    "DK",  # EK - Danimarca
    "LU",  # EL - Lussemburgo
    "NO",  # EN - Norvegia
    "PL",  # EP - Polonia
    "SE",  # ES - Svezia
    "DE",  # ET - Germania (Mil)
    "LV",  # EV - Lettonia
    "LT",  # EY - Lituania
    "AL",  # LA - Albania
    "BG",  # LB - Bulgaria
    "CY",  # LC - Cipro
    "HR",  # LD - Croazia
    "ES",  # LE - Spagna
    "FR",  # LF - Francia
    "GR",  # LG - Grecia
    "HU",  # LH - Ungheria
    "IT",  # LI - Italia
    "SI",  # LJ - Slovenia
    "CZ",  # LK - Repubblica Ceca
    "IL",  # LL - Israele
    "MT",  # LM - Malta
    "MC",  # LN - Monaco
    "AT",  # LO - Austria
    "PT",  # LP - Portogallo
    "BA",  # LQ - Bosnia ed Erzegovina
    "RO",  # LR - Romania
    "CH",  # LS - Svizzera
    "TR",  # LT - Turchia
    "MD",  # LU - Moldavia
    "PS",  # LV - Palestina
    "MK",  # LW - Macedonia del Nord
    "GI",  # LX - Gibilterra
    "RS",  # LY - Serbia
    "SK",  # LZ - Slovacchia
    "PK",  # OP - Pakistan
    "RU",  # u  - Russia
    "KZ",  # UA - Kazakistan
    "AZ",  # UB - Azerbaigian
    "AM",  # UD - Armenia
    "GE",  # UG - Georgia
    "UA",  # UK - Ucraina
    "BY",  # UM - Bielorussia
    "UZ",  # UT - Uzbekistan
    "CN",  # Cina
    "IR",  # Iran
    "AE",  # Emirati Arabi Uniti
    "KG",  # Kirghizistan
    "TJ",  # Tagikistan
    "TM",  # Turkmenistan
    "ME",  # Montenegro
    "IS",  # Islanda
    
]


In [ ]:

def find_nation(incao_code):
    airport_data = airports.get(incao_code)
    
    if airport_data:
        return airport_data.get('country')
    else:
        return None 

def load_data(path, colonne):
    DATA = []
    for data in path:
        df_temp = pd.read_csv(data, usecols = colonne)
        DATA.append(df_temp)
        
    print(len(DATA))
    df_concat = pd.concat(DATA, ignore_index=True)

    df_concat['destination_name'] = df_concat['destination'].map(find_nation,na_action='ignore')
    df_concat['origin_name'] = df_concat['origin'].map(find_nation, na_action='ignore')
    df_concat = df_concat.drop("destination", axis = 1)
    df_concat = df_concat.drop("origin", axis = 1)
    print(f"Total_row: {len(df_concat)}, added destination_name")
    return df_concat



In [ ]:
columns = ['origin', 'destination', 'callsign', 'day']

In [ ]:
#load data with incao codes assosicated at the specific country
df_21 = load_data(path_21, columns)
df_22 = load_data(path_22, columns)

In [ ]:
# df_21.to_csv(f"{path_fold_save}/df_21.csv.gz", index=False, sep=';',   compression='gzip')
# df_22.to_csv(f"{path_fold_save}/df_22.csv.gz", index=False, sep=';',   compression='gzip')  

In [ ]:
# df_21 = pd.read_csv(f"{path_fold_save}/df_21.csv.gz")
# df_22 = pd.read_csv(f"{path_fold_save}/df_21.csv.gz")

In [ ]:
def filt(data, colonna1, iso_codes):
    ris =data[(data[colonna1].isin( iso_codes)) ]
    return ris

In [ ]:
col_dest = 'destination_name'
col_orig = 'origin_name'

In [ ]:
# extract only destination_name origin_name in iso code
df_21_dest_interest  = filt(df_21, col_dest, iso_codes)
df_22_dest_interest  = filt(df_22, col_dest, iso_codes) 

In [ ]:
# df_21_dest_interest.to_csv(f"{path_fold_save}/df_21_dest_interest.csv" ,index=False, sep=';')
# df_22_dest_interest.to_csv(f"{path_fold_save}/df_22_dest_interest.csv" ,index=False, sep=';')

In [ ]:
# df_21_dest_interest = pd.read_csv(f"{path_fold_save}/df_21_dest_interest.csv.gz")
# df_22_dest_interest = pd.read_csv(f"{path_fold_save}/df_22_dest_interest.csv.gz")

In [ ]:
def monthly_destination_counts(df, day_col='day', dest_col='destination_name',
                               utc=True, period='M', as_pivot=True,
                               save_path=None, verbose=False):
    """
    Raggruppa i voli per mese (YYYY-MM) e destinazione.
    Restituisce (monthly_dest, monthly_pivot) — monthly_pivot può essere None se as_pivot=False.
    """
    df2 = df.copy()
    # extract datatime
    if not pd.api.types.is_datetime64_any_dtype(df2[day_col]):
        df2[day_col] = pd.to_datetime(df2[day_col], utc=utc, errors='coerce')
        
    # remove invalid destination or day
    df2 = df2.dropna(subset=[day_col, dest_col])
    df2['month'] = df2[day_col].dt.to_period(period).astype(str)

    monthly_dest = (df2.groupby(['month', dest_col])
                        .size()
                        .reset_index(name='flights'))

    monthly_pivot = None
    if as_pivot:
        monthly_pivot = (monthly_dest
                         .pivot(index=dest_col, columns='month', values='flights')
                         .fillna(0)
                         .astype(int))

    if save_path:
        monthly_dest.to_csv(save_path, index=False)

    return monthly_dest, monthly_pivot

In [ ]:
#extract number of destination for each country
monthly_dest_21, monthly_pivot_21 = monthly_destination_counts(df_21_dest_interest, verbose=True)
monthly_dest_22, monthly_pivot_22 = monthly_destination_counts(df_22_dest_interest, verbose=True)

#find mounth
monthly_dest_21['mese_id'] = monthly_dest_21['month'].astype(str).str[-2:]
monthly_dest_22['mese_id'] = monthly_dest_22['month'].astype(str).str[-2:]

#merge
monthly_dest_merge= pd.merge(monthly_dest_21, monthly_dest_22, on=["mese_id", "destination_name"], suffixes=("", "_2021"),       how='outer')

monthly_dest_merge = monthly_dest_merge.rename(columns={'flights': 'flights_2021', 'month': 'month_2021'})

final_monthly_dest_merge= pd.merge(monthly_dest_merge, monthly_dest_22, on=["mese_id", "destination_name"], suffixes=("", "_2022"),  how='outer')

final_monthly_dest_merge = final_monthly_dest_merge.rename(columns={'flights': 'flights_2022', 'month': 'month_2022'})

#data cleaning
cols_flights = ['flights_2021', 'flights_2022']
final_monthly_dest_merge[cols_flights] = final_monthly_dest_merge[cols_flights].fillna(0)

#drop useless columns
final_dest_df = final_monthly_dest_merge.drop(["month_2021", "month_2022","Unnamed: 0.1","Unnamed: 0"], axis=1,errors='ignore')



In [ ]:
aimCodes = ["RU","UA","PL","HU","RO","LT","SK"]

# find destination in eurasia
race_bar = final_dest_df[final_dest_df["destination_name"].isin(aimCodes)]

In [ ]:
# race_bar.to_csv(f"{path_fold_save}/race_bar.csv")